In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./part.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Vehicle_Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_tr = html.xpath('//tr[@class="detail-app-row"]')
                
                # = = = = = = = = = = = = = = =
                
                dict_vehicle = {}
                while True:
                    for tr in list_tr:
                        year = int(tr.xpath('./td/text()')[0].strip())
                        make = tr.xpath('./td/text()')[1].strip()
                        model = tr.xpath('./td/text()')[2].strip()

                        if make not in dict_vehicle:
                            dict_vehicle[make] = {model: [year]}
                        else:
                            if model not in dict_vehicle[make]:
                                dict_vehicle[make][model] = [year]
                            else:
                                dict_vehicle[make][model].append(year)
                            
                    # = = = = = = = = = = = = = = =
                    
                    if len(html.xpath('//a[@aria-label="Next"]')) == 0:
                        break
                    else:
                        resp = requests.get('https://www.dormanproducts.com' + html.xpath('//a[@aria-label="Next"]/@href')[0].strip(),
                                            headers=UA.get_User_Agent_Requests(),
                                            proxies=Proxy.get_Proxy_Requests()).text
                        soup = BeautifulSoup(resp, 'lxml')
                        html = etree.HTML(str(soup))
                        list_tr = html.xpath('//tr[@class="detail-app-row"]')
                        
                # = = = = = = = = = = = = = = =
                        
                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        year_max = max(dict_vehicle[make][model])
                        year_min = min(dict_vehicle[make][model])
                        if year_max == year_min:
                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                        else:
                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Vehicle_Url': url,
                                         'Vehicle': vehicle}])
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Vehicle_Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./vehicle_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：388

https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=134363&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：383
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=50054&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：382
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=50410&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：381
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=50429&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：380
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=132567&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：379
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=34670&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：378
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=50047&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：377
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=7797&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：376
https://www.dormanproducts.com/itemdetailapp.a

https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=84504&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：313
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=50029&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：312
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=84180&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：311
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=51699&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：310
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=73523&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：309
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=96632&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：308
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=80367&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：307
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=83314&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：306
https://www.dormanproducts.com/itemdetailapp.aspx?Produc

https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=85544&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：243
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=86091&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：242
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=144401&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：241
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=90713&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：240
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=50499&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：239
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=83308&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：238
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=49934&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：237
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=75315&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：236
https://www.dormanproducts.com/itemdetailapp.aspx?Produ

https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=50814&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：173
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=134366&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：172
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=131168&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：171
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=134350&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：170
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=132359&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：169
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=3481&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：168
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=67555&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：167
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=140296&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：166
https://www.dormanproducts.com/itemdetailapp.aspx?Pr

https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=85202&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：103
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=133947&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：102
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=131735&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：101
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=84603&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：100
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=85687&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：99
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=84445&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：98
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=133442&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：97
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=132363&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：96
https://www.dormanproducts.com/itemdetailapp.aspx?Produc

https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=84612&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：33
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=75296&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：32
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=134383&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：31
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=82835&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：30
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=49926&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：29
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=73546&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：28
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=50391&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：27
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=75316&PartType=Fuel%20Filler%20Neck  <->  [ok] - 剩余数量：26
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=752